# CHAPTER 6
# Data Loading, Storage, and File Formats
- Accessing data is a necessary first step for using most of the tools in this book.
- Focus on data input and output using **pandas**.
- Reading text files and other more efficient on-disk formats.
- Loading data from databases.
- Interacting with network sources like web APIs.

## Reading and Writing Data in Text Format
- **pandas** features a number of functions for reading tabular data as a **DataFrame** object.
- Most common are **read_csv** and **read_table**.

**TABLE: Parsing functions in pandas**

| Function                  | Description |
| :---                  |    :----    |
|read_csv| Load delimited data from a file, URL, or file-like object; use comma as default delimiter
|read_table| Load delimited data from a file, URL, or file-like object; use tab ('\t') as default delimiter
|read_fwf| Read data in fixed-width column format (i.e., no delimiters)
|read_clipboard| Version of read_table that reads data from the clipboard; useful for converting tables from web pages
|read_excel| Read tabular data from an Excel XLS or XLSX file
|read_hdf| Read HDF5 files written by pandas
|read_html| Read all tables found in the given HTML document
|read_json| Read data from a JSON (JavaScript Object Notation) string representation
|read_msgpack| Read pandas data encoded using the MessagePack binary format
|read_pickle| Read an arbitrary object stored in Python pickle format
|read_sas| Read a SAS dataset stored in one of the SAS system’s custom storage formats
|read_sql| Read the results of a SQL query (using SQLAlchemy) as a pandas DataFrame
|read_stata| Read a dataset from Stata file format
|read_feather| Read the Feather binary file format

Optional arguments of these function:
- **Indexing**: 
        Can treat one or more columns as the returned DataFrame, and whether to get column names from the file, the user, or not at all.
- **Type inference and data conversion**:
        This includes the user-defined value conversions and custom list of missing value markers.
- **Datetime parsing**:
        Includes combining capability, including combining date and time information spread over multiple columns into a single column in the result.
- **Iterating**:
        Support for iterating over chunks of very large files.
- **Unclean data issues**:
        Skipping rows or a footer, comments, or other minor things like numeric data with thousands separated by commas.

- Because real world data is messy many data loading functions (especially read_csv) have grown very complex in their options over time.
- **read_csv** has over 50 parameters
- Some of these functions, like **pandas.read_csv**, perform **type inference**, because the column data types are not part of the data format. That means you don’t necessarily have to specify which columns are numeric, integer, boolean, or string.
- Other data formats, like HDF5, Feather, and msgpack, have the data types stored in the format.

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np

In [2]:
# Check the documetation for read_csv
pd.read_csv?

In [3]:
# Use read_csv to read ex1.csv into a DataFrame
df = pd.read_csv('examples/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [4]:
# One can specify the delimeter with read_csv
pd.read_table('examples/ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [5]:
# Read file without header
pd.read_csv('examples/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [8]:
# Specify the columns names
pd.read_csv('examples/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [88]:
# Specify that the 4th column is supposed to be the index of the DataFrame
pd.read_csv('examples/ex2.csv',
            usecols=[0,2])

,1,3
0,5,7
1,9,11


In [87]:
pd.read_csv?

In [10]:
# Create a hierarchical index from multiple columns
pd.read_csv('examples/csv_mindex.csv',
            index_col=['key1', 'key2'])

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [11]:
# Consider a text file that does not have a fixed delimiter
# Fields here are separated by a variable amount of whitespace
list(open('examples/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [12]:
# Pass a regular expression as the delimiter
pd.read_table('examples/ex3.txt', sep='\s+')

# Because there was one fewer column name than the number of data rows,
# read_table infers that the first column should be the DataFrame’s index in this special case.

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [13]:
# You can skip the first, third, and fourth rows of a file with skiprows
list(open('examples/ex4.csv'))

['# hey!\n',
 'a,b,c,d,message\n',
 '# just wanted to make things more difficult for you\n',
 '# who reads CSV files with computers, anyway?\n',
 '1,2,3,4,hello\n',
 '5,6,7,8,world\n',
 '9,10,11,12,foo']

In [14]:
# Read above file with read_csv
pd.read_csv('examples/ex4.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


- Handling missing values is an important and frequently nuanced part of the file parsing process. 
- Missing data is usually either not present (empty string) or marked by some sentinel value. 
- By default, pandas uses a set of commonly occurring sentinels, such as **NA** and **NULL**.

In [15]:
# Check file with missing values
list(open('examples/ex5.csv'))

['something,a,b,c,d,message\n',
 'one,1,2,3,4,NA\n',
 'two,5,6,,8,world\n',
 'three,9,10,11,12,foo']

In [16]:
# Read file above using read_csv
result = pd.read_csv('examples/ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [17]:
# Check for missing values
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


**TABLE: Some read_csv/read_table function arguments**

| Argument                  | Description |
| :---                  |    :----    |
|path| String indicating filesystem location, URL, or file-like object
|sep or delimiter| Character sequence or regular expression to use to split fields in each row
|header| Row number to use as column names; defaults to 0 (first row), but should be None if there is no header row
|index_col| Column numbers or names to use as the row index in the result; can be a single name/number or a list of them for a hierarchical index
|names| List of column names for result, combine with header=None
|skiprows| Number of rows at beginning of file to ignore or list of row numbers (starting from 0) to skip.
|na_values| Sequence of values to replace with NA.
|comment| Character(s) to split comments off the end of lines.
|parse_dates| Attempt to parse data to datetime; False by default. If True, will attempt to parse all columns. Otherwise can specify a list of column numbers or name to parse. If element of list is tuple or list, will combine multiple columns together and parse to date (e.g., if date/time split across two columns).
|keep_date_col| If joining columns to parse date, keep the joined columns; False by default.
|converters| Dict containing column number of name mapping to functions (e.g., {'foo': f} would apply the function f to all values in the 'foo' column).
|dayfirst| When parsing potentially ambiguous dates, treat as international format (e.g., 7/6/2012 -> June 7, 2012); False by default.
|date_parser| Function to use to parse dates.
|nrows| Number of rows to read from beginning of file.
|iterator| Return a TextParser object for reading file piecemeal.
|chunksize| For iteration, size of file chunks.
|skip_footer| Number of lines to ignore at end of file.
|verbose| Print various parser output information, like the number of missing values placed in non-numeric columns.
|encoding| Text encoding for Unicode (e.g., 'utf-8' for UTF-8 encoded text).
|squeeze| If the parsed data only contains one column, return a Series.
|thousands| Separator for thousands (e.g., ',' or '.').

### Reading Text Files in Pieces
- When processing very large files or figuring out the right set of arguments to correctly process a large file, you may only want to read in a small piece of a file or iterate through smaller chunks of the file.

In [18]:
# Read a large file with read_csv
result = pd.read_csv('examples/ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [19]:
# Read a small number of rows with nrows
pd.read_csv('examples/ex6.csv', nrows=3)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G


In [20]:
# To read a file in pieces, specify a chunksize as a number of rows
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)
chunker

# The TextParser object returned by read_csv allows you to iterate over the parts of 
# the file according to the chunksize.

In [21]:
# Iterate over ex6.csv, aggregating the value counts in the 'key' column
tot = pd.Series([], dtype = 'float64')
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
    
tot = tot.sort_values(ascending=False)
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

### Writing Data to Text Format
- Data can also be exported to a delimited format.

In [80]:
# Read a csv file
data = pd.read_csv('examples/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [81]:
type(data)

pandas.core.frame.DataFrame

In [23]:
# Write data to a csv file using DataFrame’s to_csv method
data.to_csv('examples/output/out.csv')

In [83]:
pd.read_csv?

In [24]:
# Other delimiters can be used
import sys

data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [25]:
# Choose another sentinel value for missing values
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [26]:
# Disable row and column labels in the output
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [27]:
# Write only a subset of columns
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


In [28]:
# Series also have a to_csv method
dates = pd.date_range('1/1/2000', periods=7)
ts = pd.Series(np.arange(7), index=dates)
ts.to_csv('examples/output/tseries.csv')

### Working with Delimited Formats
- Most forms of tabular data from disk using functions like **pandas.read_table**.
- However, some manual processing may be necessary in some cases.
- For any file with a single-character delimiter, you can use Python’s built-in csv module.

In [29]:
# Example of malformed lines
list(open('examples/ex7.csv'))

['"a","b","c"\n', '"1","2","3"\n', '"1","2","3"\n']

In [30]:
# Import Python’s built-in csv module
import csv

# You need to open the file before passing it to csv.reader
f = open('examples/ex7.csv')
reader = csv.reader(f)

In [31]:
# Iterating through the reader like a file yields tuples of values with 
# any quote characters removed
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [32]:
# First, we read the file into a list of lines
with open('examples/ex7.csv') as f:
    lines = list(csv.reader(f))
    
lines

[['a', 'b', 'c'], ['1', '2', '3'], ['1', '2', '3']]

In [33]:
# Split the lines into the header line and the data lines
header, values = lines[0], lines[1:]

In [34]:
# Create a dictionary of data columns using a dictionary comprehension
# and the expression zip(*values), which transposes rows to columns
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

In [35]:
# Construct DataFrame from dict
pd.DataFrame.from_dict(data_dict)

,a,b,c
0,1,2,3
1,1,2,3


## JSON Data
- **JSON** (short for JavaScript Object Notation) has become one of the standard formats for sending data by HTTP request between web browsers and other applications.
- It is a much more free-form data format than a tabular text form like CSV.
- There are several Python libraries for reading and writing JSON data.

In [36]:
# Define a JSON object
obj = """
{"name": "Wes",
"places_lived": ["United States", "Spain", "Germany"],
"pet": null,
"siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
             {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

In [37]:
import json

# Convert a JSON string with json.loads
result = json.loads(obj)
type(result)

dict

In [38]:
# json.dumps converts a Python object back to JSON
asjson = json.dumps(result)

In [39]:
# You can pass a list of dicts (which were previously JSON objects) 
# to the DataFrame constructor and select a subset of the data fields

siblings = pd.DataFrame(result['siblings'], columns=['name', 'age', 'pets'])
siblings

,name,age,pets
0,Scott,30,"[Zeus, Zuko]"
1,Katie,38,"[Sixes, Stache, Cisco]"


In [40]:
# The pandas.read_json can automatically convert JSON datasets in specific 
# arrangements into a Series or DataFrame

data = pd.read_json('examples/example.json')
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [41]:
# To export data from pandas to JSON the to_json methods on Series and DataFrame
print(data.to_json())

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}


## XML and HTML: Web Scraping
- Python has many libraries for reading and writing data in the ubiquitous HTML and XML formats: **lxml**, **Beautiful Soup**, and **html5lib**.
- While **lxml** is comparatively much faster in general, the other libraries can better handle malformed HTML or XML files.
- **pandas** has a built-in function, **read_html**, which uses libraries like **lxml** and **Beautiful Soup** to automatically parse tables out of HTML files as DataFrame objects.
- The **pandas.read_html** function has a number of options, but by default it searches for and attempts to parse all tabular data contained within table tags.

In [42]:
# Read a html file with pd.read_html function
tables = pd.read_html('examples/fdic_failed_bank_list.html')

# Check the length of tables and its type
print(type(tables), 'length:', len(tables))

<class 'list'> length: 1


In [43]:
# Select the only element of the list tables
failures = tables[0]
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


### Parsing XML with lxml.objectify
- **XML** (eXtensible Markup Language) is another common structured data format supporting hierarchical, nested data with metadata.
- **XML** and **HTML** are structurally similar, but XML is more general.

**XML Example**: 
- The New York Metropolitan Transportation Authority (MTA) publishes a number of data series about its bus and train services.
- Here we’ll look at the performance data, which is contained in a set of XML files. 
- Each train or bus service has a different file (like *Performance_MNR.xml* for the Metro-North Railroad) containing monthly data as a series of XML records.

In [44]:
# Using lxml.objectify, we parse the file and get a reference to the root node of the
# XML file with getroot

from lxml import objectify

path = 'datasets/mta_perf/Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()

In [46]:
# root.INDICATOR returns a generator yielding each <INDICATOR> XML element
# For each record, we can populate a dict of tag names (like YTD_ACTUAL) to data values
# (excluding a few tags)

# Define an empty list
data = []

# Create a list with the tags we want to exclude
skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ',
               'DESIRED_CHANGE', 'DECIMAL_PLACES']

# Iterate over root.INDICATOR to create a list of dictionaries
for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [47]:
# Convert this list of dicts into a DataFrame
perf = pd.DataFrame(data)
perf.head()

,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,INDICATOR_UNIT,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,1,Service Indicators,M,%,95,96.9,95,96.9
1,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,2,Service Indicators,M,%,95,96,95,95
2,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,3,Service Indicators,M,%,95,96.3,95,96.9
3,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,4,Service Indicators,M,%,95,96.8,95,98.3
4,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,5,Service Indicators,M,%,95,96.6,95,95.8


## Binary Data Formats
- One of the easiest ways to store data (also known as serialization) efficiently is in **binary format**.
- Python has the built-in **pickle** serialization.
- pandas objects all have a **to_pickle** method that writes the data to disk in pickle format.
- **pickle** is only recommended as a short-term storage format. The problem is that it is hard to guarantee that the format will be stable over time.

In [48]:
# Read data from csv file
frame = pd.read_csv('examples/ex1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [49]:
# Save data into binary format
frame.to_pickle('examples/output/frame_pickle')

In [50]:
# Read a “pickled” object stored in a file with pandas.read_pickle
pd.read_pickle('examples/output/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


- **pandas** has built-in support for two more binary data formats: **HDF5** and **Message‐Pack**.
- And for **bcolz** =  A compressable column-oriented binary format based on the Blosc compression library.
- And for **Feather** = A cross-language column-oriented file format from the R programming community. Feather uses the Apache Arrow columnar memory format.

### Using HDF5 Format
- **HDF** in **HDF5** stands for hierarchical data format.
- It is available as a C library, and it has interfaces available in many other languages, including Java, Julia, MATLAB, and Python.
- Each **HDF5** file can store multiple datasets and supporting metadata.
- Compared with simpler formats, **HDF5** supports on-the-fly compression with a variety of compression modes, enabling data with repeated patterns to be stored more efficiently.
- HDF5 can be a good choice for working with very large datasets that don’t fit into memory, as you can efficiently read and write small sections of much larger arrays.
- While it’s possible to directly access **HDF5** files using either the **PyTables** or **h5py** libraries, **pandas** provides a high-level interface that simplifies storing Series and
DataFrame object. 
- The **HDFStore** class works like a dict and handles the low-level details.
- If you are processing data that is stored on remote servers, likeAmazon S3 or HDFS, using a different binary format designed for distributed storage like **Apache Parquet** may be more suitable.
- HDF5 is not a database. It is best suited for write-once, read-many datasets. While data can be added to a file at any time, if multiple writers do so simultaneously, the file can become corrupted.

In [51]:
# Create DataFrame
frame = pd.DataFrame({'a': np.random.randn(100)})
frame

,a
0,-1.058332
1,0.275382
2,0.430477
3,-1.164429
4,-1.109652
...,...
95,1.072447
96,-3.029190
97,1.669699
98,-0.480286


In [52]:
# Create an HDF5 file
store = pd.HDFStore('mydata.h5')

# Store the frame DataFrame in the HDF5 file 
store['obj1'] = frame

# Store a Series in the HDF5 file 
store['obj1_col'] = frame['a']

# Store the failures DataFrame in the HDF5 file
store['obj2'] = failures

In [53]:
# Print details about the HDF5 file we created
print(store.info())

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5
/obj1                frame        (shape->[100,1])
/obj1_col            series       (shape->[100])  
/obj2                frame        (shape->[547,7])


In [54]:
# Objects contained in the HDF5 file can then be retrieved with the same dict-like API
store['obj2']

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"
...,...,...,...,...,...,...,...
542,"Superior Bank, FSB",Hinsdale,IL,32646,"Superior Federal, FSB","July 27, 2001","August 19, 2014"
543,Malta National Bank,Malta,OH,6629,North Valley Bank,"May 3, 2001","November 18, 2002"
544,First Alliance Bank & Trust Co.,Manchester,NH,34264,Southern New Hampshire Bank & Trust,"February 2, 2001","February 18, 2003"
545,National State Bank of Metropolis,Metropolis,IL,3815,Banterra Bank of Marion,"December 14, 2000","March 17, 2005"


In [55]:
# HDFStore supports two storage schemas, 'fixed' and 'table' 
# 'table'  is generally slower, but it supports query operations using a special syntax

store.put('obj2', frame, format='table')

In [56]:
# Select rows from obj2 stored in 'mydata.h5'
store.select('obj2', where=['index >= 10 and index <= 15'])

,a
10,0.923002
11,-2.642540
12,0.948601
13,0.646128
14,-0.353679
15,0.552737


In [57]:
# Only when you run the close() method the data is written to disk
store.close()

In [58]:
# pandas.read_hdf function gives you a shortcut
pd.read_hdf('mydata.h5', 'obj2', where=['index < 5'])

,a
0,-1.058332
1,0.275382
2,0.430477
3,-1.164429
4,-1.109652


## Reading Microsoft Excel Files
- **pandas** also supports reading tabular data stored in Excel 2003 (and higher) files using either the ExcelFile class or **pandas.read_excel** function. 
- Internally these tools use the add-on packages **xlrd** and **openpyxl** to read XLS and XLSX files, respectively.

In [59]:
# Create an instance by passing a path to an xls or xlsx file
xlsx = pd.ExcelFile('examples/ex1.xlsx')

In [60]:
# Data stored in a sheet can then be read into DataFrame with parse
pd.read_excel(xlsx, 'Sheet1')

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [61]:
# You can also simply pass the filename to pandas.read_excel
# But if you are reading multiple sheets in a file, then it is faster to create the ExcelFile

frame = pd.read_excel('examples/ex1.xlsx', 'Sheet1')
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [62]:
# To write pandas data to Excel format, you must first create an ExcelWriter, 
# then write data to it using pandas objects’ to_excel method

writer = pd.ExcelWriter('examples/output/ex2.xlsx')
frame.to_excel(writer, 'Sheet1')
writer.save()

In [63]:
# You can also pass a file path to to_excel and avoid the ExcelWriter
frame.to_excel('examples/output/ex2.xlsx')

## Interacting with Web APIs
- Many websites have **public APIs** providing data feeds via JSON or some other format.
- An easy way to access these APIs from Python is via the requests package.

**EXAMPLE:** - find the last 30 GitHub issues for pandas on GitHub.

In [64]:
# Import necessary libraries
import requests

In [65]:
# Define the url from where to get the data
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'

# Get the data via requests.get method
resp = requests.get(url)
resp

<Response [200]>

In [66]:
# The Response object’s json method will return a dictionary containing JSON parsed
# into native Python objects

data = resp.json()

In [67]:
# Each element in data is a dictionary containing all of the data found on a GitHub
# issue page (except for the comments)

# Get the title for the first issue
data[0]['title']

'BUG: comparison of pandas NaT with datetime.date evaluating to True'

In [68]:
# We can pass data directly to DataFrame and extract fields of interest
issues = pd.DataFrame(data, columns=['number', 'title',
                                     'labels', 'state'])

issues

,number,title,labels,state
0,39151,BUG: comparison of pandas NaT with datetime.da...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
1,39150,DOC: 1.3 release notes,"[{'id': 134699, 'node_id': 'MDU6TGFiZWwxMzQ2OT...",open
2,39149,REF: implement is_exact_match,[],open
3,39148,"BUG: Cells containing ""NA"" (string) in excel i...","[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
4,39147,BUG: Assigning multiple new columns with loc f...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
5,39146,Refactor - ArrayManager overview issue,"[{'id': 49094459, 'node_id': 'MDU6TGFiZWw0OTA5...",open
6,39144,BUG: dataframe concatenation segementation fau...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
7,39143,QST:Why DataFrame.pivot() will change the orde...,"[{'id': 1954720290, 'node_id': 'MDU6TGFiZWwxOT...",open
8,39142,BUG: .dt.isocalendar().week results in the las...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
9,39141,ENH: Allow Series.apply to accept list-like an...,"[{'id': 697792067, 'node_id': 'MDU6TGFiZWw2OTc...",open


## Interacting with Databases
- In a business setting, most data may not be stored in text or Excel files. 
- **SQL-based** relational databases (such as SQL Server, PostgreSQL, and MySQL) are in wide use.
- Loading data from SQL into a DataFrame is fairly straightforward, and pandas has some functions to simplify the process.

**EXAMPLE:** - create a SQLite database using Python’s built-in sqlite3 driver.

In [69]:
# Import neccessary libraries
import sqlite3

In [70]:
# Create an empty SQL table
query = """
    CREATE TABLE test
    (a VARCHAR(20), b VARCHAR(20),
     c REAL, d INTEGER
    );"""

In [71]:
# Define the connection to the database
con = sqlite3.connect('mydata.sqlite')

# Execute the query
con.execute(query)

# Commit the changes to the database
con.commit()

In [72]:
# Insert a few rows of data into test table
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]

stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

con.executemany(stmt, data)

con.commit()

In [73]:
# Most Python SQL drivers (PyODBC, psycopg2, MySQLdb, pymssql, etc.) return a list
# of tuples when selecting data from a table
cursor = con.execute('select * from test')

# Fetch all rows
rows = cursor.fetchall()

In [75]:
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [74]:
# Column names are contained in the cursor’s description attribute
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [76]:
# You can pass the list of tuples to the DataFrame constructor + the columns names
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


### SQLAlchemy
- The **SQLAlchemy** project is a popular Python SQL toolkit that abstracts away many of the common differences between SQL databases. 
- **pandas** has a **read_sql** function that enables you to read data easily from a general SQLAlchemy connection.
- Here, we’ll connect to the same SQLite database with SQLAlchemy and read data from the table created before.

In [77]:
# Import the package
import sqlalchemy as sqla

In [78]:
# Create teh sqla engine
db = sqla.create_engine('sqlite:///mydata.sqlite')

In [79]:
# Use pandas read_sql function to read the data directly into a DataFrame
pd.read_sql('select * from test', db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [89]:
pd.read_csv?